In [1]:
cd ..

/home/hantoine/Documents/Cours/Concordia/2019-Winter/SOEN_691_Big_Data_Analytics/project/accident-prediction-montreal


In [2]:
%matplotlib inline
import sys
sys.path.insert(0, '/home/hantoine/concordia/bigdata/project/spark/python')

In [3]:
from preprocess import get_negative_samples, get_positive_samples
from utils import init_spark
spark = init_spark()
spark.version

Spark Session created
Parameters:
	spark.driver.port: 33571
	spark.driver.extraClassPath: ./data/xgboost4j-spark-0.72.jar:./data/xgboost4j-0.72.jar
	spark.network.timeout: 300s
	spark.master: local[10]
	spark.executor.id: driver
	spark.driver.host: laptop-hantoine.wireless.concordia.ca
	spark.app.name: Accident prediction
	spark.cleaner.periodicGC.interval: 5min
	spark.serializer: org.apache.spark.serializer.KryoSerializer
	spark.driver.memory: 6g
	spark.rdd.compress: True
	spark.app.id: local-1557778148297
	spark.serializer.objectStreamReset: 100
	spark.submit.pyFiles: 
	spark.submit.deployMode: client
	spark.ui.showConsoleProgress: true


'3.0.0-SNAPSHOT'

In [4]:
neg_samples = get_negative_samples(spark).sample(0.5)
pos_samples = get_positive_samples(spark)

In [6]:
imbalance_ratio = (neg_samples.count()/pos_samples.count())
print(f"Imbalance ratio: {imbalance_ratio}")

Imbalance ratio: 17.2337142814654


In [7]:
from preprocess import get_dataset_df
train_set, test_set = get_dataset_df(spark, pos_samples, neg_samples)
train_set, test_set = train_set.persist(), test_set.persist()
train_set.count()

Skip extracting road features: already done


1889517

In [7]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit, CrossValidator
from pyspark.ml import Pipeline
from random_undersampler import RandomUnderSampler
from class_weighter import ClassWeighter

rf = RandomForestClassifier(labelCol="label",
                            featuresCol="features",
                            cacheNodeIds=True,
                            maxDepth=17,
                            impurity='entropy',
                            featureSubsetStrategy='sqrt',
                             weightCol='weight',
                            minInstancesPerNode=10,
                            numTrees=100,
                            subsamplingRate=1.0,
                            maxMemoryInMB=768
                           )
ru = RandomUnderSampler().setIndexCol('id').setTargetImbalanceRatio(8.0)
cw = ClassWeighter().setClassWeight([1/imbalance_ratio, 1.0])
pipeline = Pipeline().setStages([
#                                  ru,
                                 cw,
                                 rf])
model = pipeline.fit(train_set)

In [ ]:
def print_params(model):
    for stage in model.stages:
        params = stage.extractParamMap()
        for k in params:
            print(f'{k.name}: {params[k]}')

print_params(model)

In [ ]:
predictions = model.transform(test_set).persist()

In [76]:
from evaluate import evaluate_binary_classifier
evaluate_binary_classifier(predictions)

Area Under PR = 0.37413307168449417
Area Under ROC = 0.8748248554408862


(0.37413307168449417, 0.8748248554408862)

In [ ]:
train_predictions = model.transform(train_set).persist()

In [ ]:
from evaluate import evaluate_binary_classifier
evaluate_binary_classifier(train_predictions)

In [ ]:
from random_forest import features_col
import pandas as pd
fi = pd.DataFrame(model
                  .stages[1]
                  .featureImportances
                  .toArray())
fi.index = features_col
fi.columns = ['Feature Importances']
fi = fi.sort_values(by=['Feature Importances'], ascending=False)
fi.plot.bar()

In [ ]:
from random_forest import compute_precision_recall_graph
precision_recall = compute_precision_recall_graph(predictions, 20)

In [ ]:
precision_recall.plot(x='Threshold', title='RF')

In [ ]:
precision_recall.plot(x='Recall', y='Precision', ylim=(0,1))